# I. Hiển thị ra bản đồ

In [4]:
import pandas as pd
import folium

# Đọc dữ liệu từ file CSV
data = pd.read_csv("countries.csv")

# 1. Kiểm tra dữ liệu bị thiếu
print(data.isnull().sum())

# 2. Loại bỏ các dòng bị thiếu dữ liệu
data.dropna(inplace=True)

# 3. Kiểm tra dữ liệu trùng lặp
print(data.duplicated().sum())

# 4. Loại bỏ dữ liệu trùng lặp (nếu có)
data.drop_duplicates(inplace=True)

# Tạo bản đồ với tâm là trung tâm của thế giới
world_map = folium.Map(location=[0, 0], zoom_start=2)

# Thêm marker cho mỗi quốc gia
for index, row in data.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=row['name'],
    ).add_to(world_map)

# Lưu bản đồ vào file HTML
world_map.save("world_map.html")
world_map

country      1
latitude     1
longitude    1
name         0
dtype: int64
0


# II. Áp dụng 3 thuật toán  K-Means, DBSCAN, và DPC

In [5]:
import pandas as pd
import numpy as np
import folium
from sklearn.cluster import KMeans, DBSCAN
from sklearn.neighbors import NearestNeighbors
from kneed import KneeLocator
from scipy.spatial.distance import pdist, squareform
from collections import OrderedDict

# Đọc dữ liệu từ file CSV (đã tiền xử lý)
data = pd.read_csv("countries.csv")
# 1. Kiểm tra dữ liệu bị thiếu
print(data.isnull().sum())

# 2. Loại bỏ các dòng bị thiếu dữ liệu
data.dropna(inplace=True)

# 3. Kiểm tra dữ liệu trùng lặp
print(data.duplicated().sum())

# 4. Loại bỏ dữ liệu trùng lặp (nếu có)
data.drop_duplicates(inplace=True)

coordinates = data[['latitude', 'longitude']].values

# 1. K-Means
wcss = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, random_state=42)
    kmeans.fit(coordinates)
    wcss.append(kmeans.inertia_)
knee = KneeLocator(range(1, 11), wcss, curve="convex", direction="decreasing")
optimal_k = knee.elbow

kmeans = KMeans(n_clusters=optimal_k, random_state=42)
kmeans_clusters = kmeans.fit_predict(coordinates)
data['kmeans_cluster'] = kmeans_clusters

# 2. DBSCAN
neighbors = NearestNeighbors(n_neighbors=2)
neighbors_fit = neighbors.fit(coordinates)
distances, indices = neighbors_fit.kneighbors(coordinates)
distances = np.sort(distances, axis=0)
distances = distances[:, 1]
knee = KneeLocator(range(len(distances)), distances, curve="convex", direction="increasing")
optimal_eps = knee.elbow

dbscan = DBSCAN(eps=optimal_eps, min_samples=5)
dbscan_clusters = dbscan.fit_predict(coordinates)
data['dbscan_cluster'] = dbscan_clusters

country      1
latitude     1
longitude    1
name         0
dtype: int64
0


c:\Users\GIAP\anaconda3\envs\train\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\GIAP\anaconda3\envs\train\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\GIAP\anaconda3\envs\train\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\GIAP\anaconda3\envs\train\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change fr

In [6]:
# 3. DPC
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv
from scipy.spatial.distance import pdist,squareform
from collections import OrderedDict
from itertools import combinations,product
from sklearn.cluster import SpectralClustering
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.semi_supervised import LabelPropagation
from sklearn import metrics
from sklearn import datasets
from sklearn.metrics import mean_squared_error,accuracy_score,mean_absolute_error,f1_score

def getDistCut(distList, distPercent):
    return max(distList) * distPercent / 100

def getRho(n,distMatrix,distCut):
    rho = np.zeros(n,dtype=float)
    for i in range(n-1):
        for j in range(i+1,n):
            rho[i] = rho[i] + np.exp(-(distMatrix[i, j] / distCut) ** 2)
            rho[j] = rho[j] + np.exp(-(distMatrix[i, j] / distCut) ** 2)
    return rho

def DPCA(n,distMatrix,rho,blockNum):
    rhoOrdIndex = np.flipud(np.argsort(rho))
    delta = np.zeros(n,dtype=float)
    leader = np.ones(n,dtype=int) * int(-1)
    maxdist = 0
    for ele in range(n):
        if distMatrix[rhoOrdIndex[0],ele] > maxdist:
            maxdist = distMatrix[rhoOrdIndex[0],ele]
    delta[rhoOrdIndex[0]] = maxdist
    for i in range(1,n):
        mindist = np.inf
        minindex = -1
        for j in range(i):
            if distMatrix[rhoOrdIndex[i],rhoOrdIndex[j]] < mindist:
                mindist = distMatrix[rhoOrdIndex[i],rhoOrdIndex[j]]
                minindex = rhoOrdIndex[j]
        delta[rhoOrdIndex[i]] = mindist
        leader[rhoOrdIndex[i]] = minindex
    gamma = delta * rho
    gammaOrdIdx = np.flipud(np.argsort(gamma))
    clusterIdx = np.ones(n,dtype=int) * (-1)
    for k in range(blockNum):
        clusterIdx[gammaOrdIdx[k]] = k
    for i in range(n):
        if clusterIdx[rhoOrdIndex[i]] == -1:
            clusterIdx[rhoOrdIndex[i]] = clusterIdx[leader[rhoOrdIndex[i]]]
    clusterSet = OrderedDict()
    for k in range(blockNum):
        clusterSet[k] = []
    for i in range(n):
        clusterSet[clusterIdx[i]].append(i)
    return clusterSet

def getDistanceMatrix(datas):
    N,D = np.shape(datas)
    dists = np.zeros([N,N])
    
    for i in range(N):
        for j in range(N):
            vi = datas[i,:]
            vj = datas[j,:]
            dists[i,j]= np.sqrt(np.dot((vi-vj),(vi-vj)))
    return dists

def select_dc(dists):    
    N = np.shape(dists)[0]
    tt = np.reshape(dists,N*N)
    percent = 2.0
    position = int(N * (N - 1) * percent / 100)
    dc = np.sort(tt)[position  + N]
    
    return dc


def get_density(dists,dc,method=None):
    N = np.shape(dists)[0]
    rho = np.zeros(N)
    
    for i in range(N):
        if method == None:
            rho[i]  = np.where(dists[i,:]<dc)[0].shape[0]-1
        else:
            rho[i] = np.sum(np.exp(-(dists[i,:]/dc)**2))-1
    return rho
    

def get_deltas(dists,rho):
    N = np.shape(dists)[0]
    deltas = np.zeros(N)
    nearest_neiber = np.zeros(N)

    index_rho = np.argsort(-rho)
    for i,index in enumerate(index_rho):

        if i==0:
            continue
  
        index_higher_rho = index_rho[:i]
 
        deltas[index] = np.min(dists[index,index_higher_rho])
        
        index_nn = np.argmin(dists[index,index_higher_rho])
        nearest_neiber[index] = index_higher_rho[index_nn].astype(int)
    
    deltas[index_rho[0]] = np.max(deltas)   
    return deltas,nearest_neiber
        

def find_centers_auto(rho,deltas):
    rho_threshold = (np.min(rho) + np.max(rho))/ 2
    delta_threshold  = (np.min(deltas) + np.max(deltas))/ 2
    N = np.shape(rho)[0]
    
    centers = []
    for i in range(N):
        if rho[i]>=rho_threshold and deltas[i]>delta_threshold:
            centers.append(i)
    return np.array(centers)

  
def find_centers_K(rho,deltas,K):
    rho_delta = rho*deltas
    centers = np.argsort(-rho_delta)
    return centers[:K]


def cluster_PD(rho,centers,nearest_neiber):
    K = np.shape(centers)[0]
    if K == 0:
        print("can not find centers")
        return
    
    N = np.shape(rho)[0]
    labs = -1*np.ones(N).astype(int)
    

    for i, center in enumerate(centers):
        labs[center] = i
   

    index_rho = np.argsort(-rho)
    for i, index in enumerate(index_rho):

        if labs[index] == -1:
            labs[index] = labs[int(nearest_neiber[index])]
    return labs
        
def draw_decision(rho,deltas,name="1.jpg"):       
    plt.cla()
    for i in range(np.shape(datas)[0]):
        plt.scatter(rho[i],deltas[i],s=16.,color=(0,0,0))
        plt.annotate(str(i), xy = (rho[i], deltas[i]),xytext = (rho[i], deltas[i]))
        plt.xlabel("rho")
        plt.ylabel("deltas")
    plt.savefig(name)
    plt.show()

def draw_cluster(datas,labs,centers, dic_colors, name="1.jpg"):     
    plt.cla()
    K = np.shape(centers)[0]
    
    for k in range(K):
        sub_index = np.where(labs == k)
        sub_datas = datas[sub_index]
        plt.scatter(sub_datas[:,0],sub_datas[:,1],s=16.,color=dic_colors[k])
        plt.scatter(datas[centers[k],0],datas[centers[k],1],color="k",marker="+",s = 200.)
    plt.savefig(name)
    plt.show()

def dpc(coordinates, K=None):
    """Thực hiện phân cụm Density Peaks Clustering (DPC)."""
    dists = getDistanceMatrix(coordinates)
    dc = select_dc(dists)
    rho = get_density(dists, dc, method="Gaussion")
    deltas, nearest_neiber = get_deltas(dists, rho)

    if K is None:
        centers = find_centers_auto(rho, deltas)
    else:
        centers = find_centers_K(rho, deltas, K)

    labs = cluster_PD(rho, centers, nearest_neiber)
    return labs

# Thực hiện phân cụm DPC
dpc_clusters = dpc(coordinates, K=3)
data['dpc_cluster'] = dpc_clusters

In [7]:
data

,country,latitude,longitude,name,kmeans_cluster,dbscan_cluster,dpc_cluster
0,AD,42.546245,1.601554,Andorra,0,0,0
1,AE,23.424076,53.847818,United Arab Emirates,0,0,0
2,AF,33.939110,67.709953,Afghanistan,0,0,0
3,AG,17.060816,-61.796428,Antigua and Barbuda,2,0,1
4,AI,18.220554,-63.068615,Anguilla,2,0,1
...,...,...,...,...,...,...,...
240,YE,15.552727,48.516388,Yemen,0,0,0
241,YT,-12.827500,45.166244,Mayotte,0,0,0
242,ZA,-30.559482,22.937506,South Africa,0,0,0
243,ZM,-13.133897,27.849332,Zambia,0,0,0


## Hiển thị kết quả

In [10]:
colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 
              'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple']

### 1. Bản đồ K-means

In [11]:
# Bản đồ K-Means
kmeans_map = folium.Map(location=[0, 0], zoom_start=2)
for index, row in data.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=row['name'],
        icon=folium.Icon(color=colors[row['kmeans_cluster']]),
    ).add_to(kmeans_map)
kmeans_map.save("kmeans_map.html")
kmeans_map

### 2. Bản đồ DBSCAN

In [12]:
# Bản đồ DBSCAN
dbscan_map = folium.Map(location=[0, 0], zoom_start=2)
for index, row in data.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=row['name'],
        icon=folium.Icon(color=colors[row['dbscan_cluster']]),
    ).add_to(dbscan_map)
dbscan_map.save("dbscan_map.html")
dbscan_map

### 3. Bản đồ DPC

In [14]:
# Bản đồ DPC
dpc_map = folium.Map(location=[0, 0], zoom_start=2)
for index, row in data.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=row['name'],
        icon=folium.Icon(color=colors[row['dpc_cluster']]),
    ).add_to(dpc_map)
dpc_map.save("dpc_map.html")
dpc_map

# III. So sánh kết quả
**1. K-Means:**

- K-Means chia dữ liệu thành các cụm có hình dạng tròn và kích thước tương đối đồng đều.
- Thuật toán này hoạt động tốt khi dữ liệu có phân bố cụm rõ ràng và khoảng cách giữa các cụm là tương đối lớn.
- Trong trường hợp này, K-Means tạo ra 3 cụm, tập trung chủ yếu ở Châu Mỹ, Châu Phi và Châu Á - Thái Bình Dương.
- Nhược điểm của K-Means là nó bị ảnh hưởng bởi các outlier và giả định rằng các cụm có hình dạng tròn, điều này có thể không phù hợp với dữ liệu địa lý.

**2. DBSCAN:**

- DBSCAN xác định các cụm dựa trên mật độ của các điểm dữ liệu. 
- Thuật toán này có khả năng phát hiện các cụm có hình dạng bất kỳ và xử lý outlier hiệu quả.
- Trong trường hợp này, DBSCAN chỉ xác định được một cụm duy nhất (màu đỏ). Điều này cho thấy rằng hầu hết các quốc gia có mật độ phân bố tương đối đồng đều trên toàn cầu.
- DBSCAN có thể gặp khó khăn khi dữ liệu có mật độ thay đổi nhiều hoặc khi việc xác định tham số eps tối ưu là khó khăn.

**3. DPC:**

- DPC cũng là thuật toán phân cụm dựa trên mật độ, nhưng nó tập trung vào việc xác định các "đỉnh" mật độ (density peaks) làm tâm cụm. 
- Thuật toán này có khả năng phát hiện các cụm có hình dạng bất kỳ và phân biệt các vùng có mật độ khác nhau hiệu quả.
- Trong trường hợp này, DPC tạo ra 3 cụm với sự phân bố rõ ràng hơn so với K-Means. Cụm màu xanh lá cây bao gồm các quốc gia ở Châu Mỹ, cụm màu xanh dương bao gồm các quốc gia ở Châu Phi và một phần Châu Âu, cụm màu đỏ bao gồm các quốc gia ở Châu Á - Thái Bình Dương và phần còn lại của Châu Âu. 

**Kết luận:**

- Cả 3 thuật toán đều cho kết quả phân cụm hợp lý, nhưng DPC dường như tạo ra các cụm có ý nghĩa địa lý rõ ràng hơn so với K-Means và DBSCAN.
- Việc lựa chọn thuật toán phù hợp nhất phụ thuộc vào đặc điểm của dữ liệu và mục tiêu phân tích cụ thể. 